<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/LSTM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%83%9D%EC%84%B1AI_%EB%AA%A8%EB%8D%B8%EB%A7%81_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [ ]:
!pip install konlpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터 불러오기

In [ ]:
yo_review =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['고객리뷰']
yo_reply = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['사장댓글']

train = pd.concat([yo_review, yo_reply], axis=1)
train.head()

,고객리뷰,사장댓글
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [ ]:
rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [ ]:
shuffled_df = train.sample(frac=1, random_state=2)  # frac=1은 모든 행을 선택, random_state는 재현 가능한 랜덤 설정

train = shuffled_df.iloc[:1000].reset_index(drop=True)

## 데이터 정제

- 불필요한 문자 삭제

In [ ]:
review_list = []
for review in train['review']:
    if isinstance(review, str):
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9\(\)\{\}\[\]]")
        # pat = re.compile("[^a-zA-Z가-힣0-9\(\)\{\}\[\]]")
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        # pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

,review,reply
0,왼쪽은 전복죽 먹다남은거찍어서 양도푸짐하고 맛도끝내줍니다 서비스계란찜도 넘맛있어요,정말 반가워요-!!\n신경 가득 써서 준비했는데 \n이렇게 칭찬해 주시니 \n정말 ...
1,맛있게 잘 먹었습니다 감사합니다,안녕하세요.삽교원조두리곱창입니다.🐷\n저희 음식이 입맛에 맞으셨다니 정말 다행이고 ...
2,너무 맛있고 같이 주신 스티커들도 너무 귀여워요 감사합니다,감사합니다👍👍\n항상 맛있는 샐러드 전달드리겠습니다😁🙏
3,늘 맛있어요 맨날 같은 거 먹다가 하나더 주문했어요 오늘은 오리도 땡겨서요 둘다 너...,항상 좋은 식재료로 맛있는 음식전달드릴려고 노력하고있습니다ㅎㅎ\n맛았게 드셨다니 오...
4,족발은 언제나 잭아저씨,잭아저씨 족발보쌈을 찾아주셔서 감사합니다😄😄\n고객님의 리뷰가 제겐 오늘 하루를 \...


In [ ]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9\(\)\{\}\[\]]")
        # pat = re.compile("[^a-zA-Z가-힣0-9\(\)\{\}\[\]]")
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        # pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

,review,reply
0,왼쪽은 전복죽 먹다남은거찍어서 양도푸짐하고 맛도끝내줍니다 서비스계란찜도 넘맛있어요,정말 반가워요 신경 가득 써서 준비했는데 이렇게 칭찬해 주시니 정말 행복합니다 언제...
1,맛있게 잘 먹었습니다 감사합니다,안녕하세요 삽교원조두리곱창입니다 저희 음식이 입맛에 맞으셨다니 정말 다행이고 뿌듯합...
2,너무 맛있고 같이 주신 스티커들도 너무 귀여워요 감사합니다,감사합니다 항상 맛있는 샐러드 전달드리겠습니다
3,늘 맛있어요 맨날 같은 거 먹다가 하나더 주문했어요 오늘은 오리도 땡겨서요 둘다 너...,항상 좋은 식재료로 맛있는 음식전달드릴려고 노력하고있습니다 맛았게 드셨다니 오늘도 ...
4,족발은 언제나 잭아저씨,잭아저씨 족발보쌈을 찾아주셔서 감사합니다 고객님의 리뷰가 제겐 오늘 하루를 열심히 ...


### "~~님" → "고객님"으로 변환

In [ ]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)

- 정제된 데이터 저장하기

In [ ]:
# train.to_csv(f"{DATA_PATH}yogiyo_1000.csv", index=False)

- 정제된 데이터 불러오기

In [ ]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_1000.csv")

### 가게명 키워드 컬럼화(나중에 필요할 때 사용)

In [ ]:
# ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]

# store_name = []
# # region_name = []

# for store in ygy_store_lst:
#     pat = re.split(r'[- &]', store)
#     pat = ''.join(pat)
#     store_name.append(pat)
#     # if len(pat) > 1 and pat[1] and not pat[1].isdigit():
#     #     region_name.append(pat[1])

In [ ]:
# store_name[:8]

In [ ]:
# short_name = []
# for name in store_name:
#     for length in [2, 3, 4]:
#         cut = name[:length]
#         short_name.append(cut)
#     if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
#         cut = name[:6]
#         short_name.append(cut)

# short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
# short_name = list(set(short_name))
# print(len(short_name), short_name)

In [ ]:
# # short_name 에서 불용어 처리
# drop_name = ['OK','갈비','감성','걸작','고기','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교','손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지', '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',]
# short_name = [col for col in short_name if col not in drop_name]
# print(len(short_name),short_name)

In [ ]:
# train['store'] = ""

# # 각 가게명에서 가능한 단축된 형태를 확인하고 store 컬럼에 추가
# for name in short_name:
#     train[name] = train['reply'].apply(lambda x: name if name in x else '')
#     train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1)

# # 첫 번째 문자가 콤마인 경우 제거
# train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x)

# # 불필요한 컬럼 삭제
# train = train.drop(columns=short_name)

# train.head()

In [ ]:
# train[train['reply'].str.contains(short_name[2])].index

In [ ]:
# train['store'][10216]

# 한국어 형태소 분석

In [ ]:
review, reply = train["review"] , train["reply"]
len(review) , len(reply)

(1000, 1000)

In [ ]:
review[:5]

0       왼쪽은 전복죽 먹다남은거찍어서 양도푸짐하고 맛도끝내줍니다 서비스계란찜도 넘맛있어요 
1                                    맛있게 잘 먹었습니다 감사합니다
2                    너무 맛있고 같이 주신 스티커들도 너무 귀여워요 감사합니다 
3    늘 맛있어요 맨날 같은 거 먹다가 하나더 주문했어요 오늘은 오리도 땡겨서요 둘다 너...
4                                        족발은 언제나 잭아저씨 
Name: review, dtype: object

In [ ]:
import unicodedata

def normalize(sentence):
    return unicodedata.normalize('NFC', sentence)

normalize(review[10])

'몇번 시켜먹었는데 역시 맛있네요 '

In [ ]:
from konlpy.tag import Mecab, Okt

def clean_text(sentence):
    okt = Okt()
    sentence = normalize(sentence)
    sentence = okt.morphs(sentence)
    sentence = ' '.join(sentence)
    sentence = sentence.lower()
    return sentence

clean_text(review[0])

'왼쪽 은 전복죽 먹다 남은거 찍어서 양도 푸 짐 하고 맛 도 끝내줍니다 서비스 계란찜 도 넘 맛있어요'

In [ ]:
review = [clean_text(sent) for sent in review.values]
reply = [clean_text(sent) for sent in reply.values]

- 형태소 분석 완료 데이터 저장

In [ ]:
# import pickle
# with open(f'{DATA_PATH}cleanText_data.pkl', 'wb') as file:
#     data_dict = {'review':review, 'reply':reply}
#     pickle.dump(data_dict, file)

### 형태소 분석 완료 데이터 불러오기

In [ ]:
import pickle
with open(f'{DATA_PATH}cleanText_data.pkl', 'rb') as file:
    data_dict = pickle.load(file)

# 불러온 데이터 사용 예시
review = data_dict['review']
reply = data_dict['reply']

In [ ]:
review[:5]

['왼쪽 은 전복죽 먹다 남은거 찍어서 양도 푸 짐 하고 맛 도 끝내줍니다 서비스 계란찜 도 넘 맛있어요',
 '맛있게 잘 먹었습니다 감사합니다',
 '너무 맛있고 같이 주신 스티커 들 도 너무 귀여워요 감사합니다',
 '늘 맛있어요 맨날 같은 거 먹다가 하나 더 주문 했어요 오늘 은 오리 도 땡겨서요 둘다 너무 맛있어요',
 '족발 은 언제나 잭 아저씨']

In [ ]:
reply[:5]

['정말 반가워요 신경 가득 써서 준비 했는데 이렇게 칭찬 해 주시 니 정말 행복합니다 언제나 만족 스럽게 드실 수 있도록 세 심히 살피겠습니다 시원하고 상쾌한 하루 보내세요',
 '안녕하세요 삽 교원 조 두리 곱창 입니다 저희 음식 이 입맛 에 맞으셨다니 정말 다행 이고 뿌듯합니다 코로나 로 인해 어려운 시기 에 찾아주셔서 감사합니다 맛있게 드신거 같아 감사합니다 다음주 문도 찾아주시는 그날 까지 더욱 더 노력 하겠습니다 항상 당일 납품 받는 재료 로 최고 의 신선도 를 유지 하여 맛있는 음식 을 대접 하기 위해 더욱 더 노력 하는 삽 교원 조 두리 곱창 이 되겠습니다 다시 한번 주문 해주셔 감사합니다',
 '감사합니다 항상 맛있는 샐러드 전달 드리겠습니다',
 '항상 좋은 식 재료 로 맛있는 음식 전달 드릴려고 노력 하고있습니다 맛았 게 드셨다니 오늘 도 감사합니다',
 '잭 아저씨 족발 보쌈 을 찾아주셔서 감사합니다 고객 님 의 리뷰 가 제겐 오늘 하루 를 열심히 달릴 수 있게 해주는 활력소 에요 앞 으로도 칭찬 리뷰 가 끊기지 않도록 최선 을 다해 맛있는 음식 을 대접 드리겠습니다 늘 믿고 주문 하실 수 있는 최고 의 매장 이 되기 위해 힘낼게요 바쁜 시간 내셔서 리뷰 남겨주신 점 정말 감사합니다 다시 한번 저희 매장 찾아주시면 그때 도 후회 없으실 것 을 약속 드립니다 저희 매장 을 이용 하시고 만족하셨다면 상단 에 찜 을 눌러주세요 다음 주 문 시 에 편리하게 주문 하실 수 있습니다 항상 건강하시고 행복하세요']

# 단어 사전 만들기

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

In [ ]:
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2

class WordVocab():
    def __init__(self):
        self.word2index = {'<PAD>': PAD_TOKEN,
                           '<SOS>': SOS_TOKEN,
                           '<EOS>': EOS_TOKEN,}
        self.word2count = {}
        self.index2word = {PAD_TOKEN: '<PAD>',
                           SOS_TOKEN: '<SOS>',
                           EOS_TOKEN: '<EOS>'}
        self.n_words = 3  # PAD, SOS, EOS 포함

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
print(f'원문: {review[0]}')
lang = WordVocab()
lang.add_sentence(review[0])
print('단어사전: ', lang.word2index)

원문: 왼쪽 은 전복죽 먹다 남은거 찍어서 양도 푸 짐 하고 맛 도 끝내줍니다 서비스 계란찜 도 넘 맛있어요
단어사전:  {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '왼쪽': 3, '은': 4, '전복죽': 5, '먹다': 6, '남은거': 7, '찍어서': 8, '양도': 9, '푸': 10, '짐': 11, '하고': 12, '맛': 13, '도': 14, '끝내줍니다': 15, '서비스': 16, '계란찜': 17, '넘': 18, '맛있어요': 19}


In [ ]:
max_length = 10
sentence_length = 6

sentence_tokens = np.random.randint(low=3, high=100, size=(sentence_length,))
sentence_tokens = sentence_tokens.tolist()
print(f'Generated Sentence: {sentence_tokens}')

sentence_tokens = sentence_tokens[:(max_length-1)]

token_length = len(sentence_tokens)

# 문장의 맨 끝부분에 <EOS> 토큰 추가
sentence_tokens.append(2)

for i in range(token_length, max_length-1):
    # 나머지 빈 곳에 <PAD> 토큰 추가
    sentence_tokens.append(0)

print(f'Output: {sentence_tokens}')
print(f'Total Length: {len(sentence_tokens)}')

Generated Sentence: [59, 3, 33, 41, 60, 15]
Output: [59, 3, 33, 41, 60, 15, 2, 0, 0, 0]
Total Length: 10


# 데이터셋 만들기

In [ ]:
import unicodedata

def normalize(sentence):
    return unicodedata.normalize('NFC', sentence)

normalize(review[10])

'몇번 시켜먹었는데 역시 맛있네요 '

In [ ]:
from konlpy.tag import Mecab, Okt

def clean_text(sentence):
    okt = Okt()
    sentence = normalize(sentence)
    sentence = okt.morphs(sentence)
    sentence = ' '.join(sentence)
    sentence = sentence.lower()
    return sentence

clean_text(review[0])

'왼쪽 은 전복죽 먹다 남은거 찍어서 양도 푸 짐 하고 맛 도 끝내줍니다 서비스 계란찜 도 넘 맛있어요'

In [ ]:
class TextDataset(Dataset):
    def __init__(self, df, min_length=3, max_length=32):
        super(TextDataset, self).__init__()
        data_dir = 'data'

        # TOKEN 정의
        self.PAD_TOKEN = 0 # Padding 토큰
        self.SOS_TOKEN = 1 # SOS 토큰
        self.EOS_TOKEN = 2 # EOS 토큰

        self.tagger = Okt()   # 형태소 분석기
        self.max_length = max_length # 한 문장의 최대 길이 지정

        # 한글 정규화
        korean_pattern = r'[^a-zA-Z가-힣0-9]'
        self.normalizer = re.compile(korean_pattern)

        # src: 질의, tgt: 답변
        src_clean = []
        tgt_clean = []

        # 단어 사전 생성
        wordvocab = WordVocab()

        for _, row in df.iterrows():
            src = row['review']
            tgt = row['reply']

            # 한글 전처리
            src = self.clean_text(src)
            tgt = self.clean_text(tgt)

            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)

        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab

    def normalize(sentence):
        # 정규표현식에 따른 한글 정규화
        return unicodedata.normalize('NFC', sentence)

    def clean_text(self, sentence):
        # 한글 정규화
        sentence = normalize(sentence)
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence

    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]

    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)

        # 문장의 맨 끝부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS_TOKEN)

        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳에 <PAD> 토큰 추가
            sentence_tokens.append(self.PAD_TOKEN)
        return sentence_tokens

    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs)
        inputs_padded = self.pad_sequence(inputs_sequences)

        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)

    def __len__(self):
        return len(self.srcs)

In [ ]:
MAX_LENGTH = 25

dataset = TextDataset(train, min_length=3, max_length=MAX_LENGTH)

In [ ]:
x, y = dataset[0]

In [ ]:
print(f'x shape: {x.shape}')
print(x)

x shape: torch.Size([25])
tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 14, 18, 19,
         2,  0,  0,  0,  0,  0,  0])


In [ ]:
print(f'y shape: {y.shape}')
print(y)

y shape: torch.Size([25])
tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 20, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42,  2])


## 데이터셋 분할

In [ ]:
# 80%의 데이터를 train에 할당합니다.
train_size = int(len(dataset) * 0.8)
train_size

606

In [ ]:
# 나머지 20% 데이터를 test에 할당합니다.
test_size = len(dataset) - train_size
test_size

152

In [ ]:
from torch.utils.data import random_split

# 랜덤 스플릿으로 분할을 완료합니다.
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 데이터로더 만들기

In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# 1개의 배치 데이터를 추출
x, y = next(iter(train_loader))

# shape: (batch_size, sequence_length)
x.shape, y.shape

(torch.Size([16, 25]), torch.Size([16, 25]))

# Encoder 만들기

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        super(Encoder, self).__init__()

        # 단어 사전의 개수 지정
        self.num_vocabs = num_vocabs
        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        # GRU (embedding dimension)
        self.gru = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)
        output, hidden = self.gru(x)
        return output, hidden

In [ ]:
embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

# x의 shape을 변경합니다.
# (batch_size, sequence_length) => (sequence_length, batch_size)
embedded = embedding(x)

print(x.shape)
print(embedded.shape)
# input:  (sequence_length, batch_size)
# output: (sequence_length, batch_size, embedding_dim)

torch.Size([16, 25])
torch.Size([16, 25, 64])


In [ ]:
embedded = embedded.permute(1, 0, 2)
print(embedded.shape)
# (sequence_length, batch_size, embedding_dim)

torch.Size([25, 16, 64])


In [ ]:
hidden_size = 32

gru = nn.GRU(embedding_dim,      # embedding 차원
             hidden_size,
             num_layers=1,
             bidirectional=False)

# input       : (sequence_length, batch_size, embedding_dim)
# h0          : (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
o, h = gru(embedded, None)

print(o.shape)
print(h.shape)
# output      : (sequence_length, batch_size, hidden_size x bidirectional(1))
# hidden_state: (bidirectional(1) x number of layers(1), batch_size, hidden_size)

torch.Size([25, 16, 32])
torch.Size([1, 16, 32])


In [ ]:
NUM_VOCABS = dataset.wordvocab.n_words
print(f'number of vocabs: {NUM_VOCABS}')

number of vocabs: 5280


In [ ]:
# Encoder 정의
encoder = Encoder(NUM_VOCABS, hidden_size=32, embedding_dim=64, num_layers=1)

## Encoder 정의

In [ ]:
# Encoder에 x 통과 후 output, hidden_size 의 shape 확인
# input(x)    : (batch_size, sequence_length)
o, h = encoder(x)

print(o.shape)
print(h.shape)
# output      : (sequence_length, batch_size, hidden_size x bidirectional(1))
# hidden_state: (bidirectional(1) x number of layers(1), batch_size, hidden_size)

torch.Size([25, 16, 32])
torch.Size([1, 16, 32])


# Decoder 만들기

In [ ]:
class Decoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super(Decoder, self).__init__()
        # 단어사전 개수
        self.num_vocabs = num_vocabs
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

        # 최종 출력은 단어사전의 개수
        self.fc = nn.Linear(hidden_size, num_vocabs)

    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        embedded = F.relu(self.embedding(x))
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden_state)
        output = self.fc(output.squeeze(0)) # (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
        return output, hidden

In [ ]:
x = torch.abs(torch.randn(size=(1, 16)).long())
print(x)
x.shape
# batch_size = 16 이라 가정했을 때,
# (1, batch_size)
# 여기서 batch_size => (1, batch_size) 로 shape 변환을 선행

tensor([[1, 0, 0, 0, 0, 0, 1, 3, 1, 1, 0, 0, 0, 1, 1, 1]])


torch.Size([1, 16])

In [ ]:
embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

embedded = embedding(x)
embedded.shape
# embedding 출력
# (1, batch_size, embedding_dim)

torch.Size([1, 16, 64])

In [ ]:
hidden_size = 32

gru = nn.GRU(embedding_dim,
             hidden_size,
             num_layers=1,
             bidirectional=False,
             batch_first=False, # batch_first=False로 지정
            )

o, h = gru(embedded)

print(o.shape)
# output shape: (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
print(h.shape)
# hidden_state shape: (Bidirectional(1) x number of layers(1), batch_size, hidden_size(32))

torch.Size([1, 16, 32])
torch.Size([1, 16, 32])


In [ ]:
fc = nn.Linear(32, NUM_VOCABS) # 출력은 단어사전의 개수로 가정

output = fc(o[0])

print(o[0].shape)
print(output.shape)
# input : (batch_size, output from GRU)
# output: (batch_size, output dimension)

torch.Size([16, 32])
torch.Size([16, 5280])


## Decoder 정의

In [ ]:
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words, hidden_size=32, embedding_dim=64, num_layers=1)

In [ ]:
x, y = next(iter(train_loader))

o, h = encoder(x)

print(o.shape, h.shape)
# output      : (batch_size, sequence_length, hidden_size(32) x bidirectional(1))
# hidden_state: (Bidirectional(1) x number of layers(1), batch_size, hidden_size(32))

torch.Size([25, 16, 32]) torch.Size([1, 16, 32])


In [ ]:
x = torch.abs(torch.full(size=(16,), fill_value=SOS_TOKEN).long())
print(x)
x.shape
# batch_size = 16 이라 가정(16개의 SOS 토큰)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


torch.Size([16])

In [ ]:
decoder_output, decoder_hidden = decoder(x, h)
decoder_output.shape, decoder_hidden.shape
# (batch_size, num_vocabs), (1, batch_size, hidden_size)

(torch.Size([16, 5280]), torch.Size([1, 16, 32]))

# Seq2Seq 모델 만들기

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        # inputs : (batch_size, sequence_length)
        # outputs: (batch_size, sequence_length)

        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_vocabs

        # 리턴할 예측된 outputs를 저장할 임시 변수
        # (sequence_length, batch_size, num_vocabs)
        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)

        # 인코더에 입력 데이터 주입, encoder_output은 버리고 hidden_state 만 살립니다.
        # 여기서 hidden_state가 디코더에 주입할 context vector 입니다.
        # (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
        _, decoder_hidden = self.encoder(inputs)

        # (batch_size) shape의 SOS TOKEN으로 채워진 디코더 입력 생성
        decoder_input = torch.full((batch_size,), SOS_TOKEN, device=self.device)

        # 순회하면서 출력 단어를 생성합니다.
        # 0번째는 SOS TOKEN이 위치하므로, 1번째 인덱스부터 순회합니다.
        for t in range(0, output_length):
            # decoder_input : 디코더 입력 (batch_size) 형태의 SOS TOKEN로 채워진 입력
            # decoder_output: (batch_size, num_vocabs)
            # decoder_hidden: (Bidirectional(1) x number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output

            # teacher forcing 적용 여부 확률로 결정
            # teacher forcing 이란: 정답치를 다음 RNN Cell의 입력으로 넣어주는 경우. 수렴속도가 빠를 수 있으나, 불안정할 수 있음
            teacher_force = random.random() < teacher_forcing_ratio

            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1)

            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, num_vocabs)로 변경

In [ ]:
# Encoder 정의
encoder = Encoder(num_vocabs=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Seq2Seq 정의
seq2seq = Seq2Seq(encoder, decoder, 'cpu')

In [ ]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# (batch_size, sequence_length), (batch_size, sequence_length)

torch.Size([16, 25]) torch.Size([16, 25])


In [ ]:
output = seq2seq(x, y)
print(output.shape)
# (batch_size, sequence_length, num_vocabs)

torch.Size([16, 25, 5280])


# 학습하기

In [ ]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_SIZE = 512
EMBEDDIMG_DIM = 256

print(f'num_vocabs: {NUM_VOCABS}\n======================')

# Encoder 정의
encoder = Encoder(num_vocabs=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

num_vocabs: 5280
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5280, 256)
    (gru): GRU(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(5280, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (gru): GRU(256, 512)
    (fc): Linear(in_features=512, out_features=5280, bias=True)
  )
)


# 하이퍼파라미터 정의

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0

        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta


    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')

        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')


        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [ ]:
LR = 1e-3
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
es = EarlyStopping(patience=5, delta=0.001, mode='min', verbose=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

# 학습 loop 만들기

In [ ]:
def train(model, data_loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0

    for x, y in data_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        # output: (batch_size, sequence_length, num_vocabs)
        output = model(x, y)
        output_dim = output.size(2)

        # 1번 index 부터 슬라이싱한 이유는 0번 index가 SOS TOKEN 이기 때문
        # (batch_size*sequence_length, num_vocabs) 로 변경
        output = output.reshape(-1, output_dim)

        # (batch_size*sequence_length) 로 변경
        y = y.view(-1)

        # Loss 계산
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(data_loader)

# 테스트 loop 만들기

In [ ]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()

    eval_loss = 0

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)

            # Loss 계산
            loss = loss_fn(output, y)

            eval_loss += loss.item() * x.size(0)

    return eval_loss / len(data_loader)

# 추론 함수

In [ ]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS_TOKEN, EOS_TOKEN, PAD_TOKEN]:
            outputs.append(word)
        if word == EOS_TOKEN:
            break
    return ' '.join(outputs)

# 챗봇 함수화

In [ ]:
def random_evaluation(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            for i in range(n):
                print(f'질문   : {sequence_to_sentence(x[i], index2word)}')
                print(f'답변   : {sequence_to_sentence(y[i], index2word)}')
                print(f'예측답변: {sequence_to_sentence(preds[i].argmax(1), index2word)}')
                print('==='*10)

In [ ]:
NUM_EPOCHS = 20
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/seq2seq-chatbot-kor.pt'

best_loss = np.inf

for epoch in tqdm(range(NUM_EPOCHS), total=NUM_EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn, device)

    val_loss = evaluate(model, test_loader, loss_fn, device)

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)

    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

    # Early Stop
    es(loss)
    if es.early_stop:
        break

    # Scheduler
    scheduler.step(val_loss)

model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'/content/drive/MyDrive/project_final/data/models/seq2seq-chatbot-kor-{best_loss:.4f}.pt')

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1, loss: 35.0493, val_loss: 76.9781
[EarlyStopping] (Patience) 1/5, Best: 30.85113, Current: 35.04931, Delta: 4.19818
[EarlyStopping] (Patience) 2/5, Best: 30.85113, Current: 32.35160, Delta: 1.50047
[EarlyStopping] (Patience) 3/5, Best: 30.85113, Current: 32.28246, Delta: 1.43133
Epoch 00023: reducing learning rate of group 0 to 1.2500e-04.
[EarlyStopping] (Update) Best Score: 30.29710
[EarlyStopping] (Patience) 1/5, Best: 30.29710, Current: 30.44038, Delta: 0.14328
epoch: 6, loss: 30.7028, val_loss: 76.9470
[EarlyStopping] (Patience) 2/5, Best: 30.29710, Current: 30.70282, Delta: 0.40572
Epoch 00026: reducing learning rate of group 0 to 6.2500e-05.
[EarlyStopping] (Update) Best Score: 29.45390
[EarlyStopping] (Update) Best Score: 28.03338
[EarlyStopping] (Update) Best Score: 28.00966
Epoch 00029: reducing learning rate of group 0 to 3.1250e-05.
[EarlyStopping] (Patience) 1/5, Best: 28.00966, Current: 28.53166, Delta: 0.52200
epoch: 11, loss: 27.6691, val_loss: 78.3827
[EarlySt

# 결과 확인하기

In [ ]:
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 언제나 만족스럽습니다 잘 먹었습니다
답변   : 안녕하세요 만족스런 식사 가 되셨다니 정말 기쁘네요 불맛 이 확 들어간 양념 석쇠 불고기 를 사랑 해주셔서 정말 감사 드립니다 든든한 한 끼 식사
예측답변: 고객 님 안녕하세요 항상 고객 님 께 최선 을 다 하는 뜸 들 이다 북서울꿈의숲 점 입니다 소중한 별 점 5 점 리뷰 감사합니다
질문   : 최근 들어 자주 시켜 먹고있는데 오늘 유독 오이 가 너무 써서 처음 으로 오이 를 다 못 먹었네요 이전 에 시켜 먹었을 때 는
답변   : 고객 님 안녕하세요 소중한 식사 시간 에 불편 드려 정말 죄송합니다 오이 가 식 자재 에서 바로 받아 온건데 쓴 오이 들이 많아 우선
예측답변: 안녕하세요 주문 해주셔서 감사합니다 드립니다 아무래도 씬 도우 이 마음 들 은 늘 푸 짐 한 양 과 맛 으로 보답 하겠습니다
질문   : 언제나 맛있네요 쫄깃 하고 부드럽고
답변   : 식사 는 맛있게 하셨나요 혜화 대학로 sns 피자 랭킹 맛집 피자 보이시 나 대학로 점 을 믿고 주문 해주셔서 정말 감사합니다 저희 피자 보이시나는
예측답변: 고객 님 안녕하세요 시간 내어 리뷰 작성 해주셔서 정말 감사합니다 항상 고객 님 들 주문 하나 하나 에 정성 을 담아 대접 하겠습니다 믿고
질문   : 냉면 맛 나 요 물만두 는 떡 이 돼서 왔어요
답변   : 고객 님 안녕하세요 소중한 식사 시간 에 불편 드려 정말 죄송합니다 물만두 는 최대한 떡 이 되지 않게 마지막 으로 포장 하고 있는데 더욱
예측답변: 고객 님 안녕하세요 시간 내어 리뷰 작성 해주셔서 정말 감사합니다 항상 고객 님 들 주문 하나 하나 에 정성 을 담아 대접 하겠습니다 믿고
질문   : 맛있게 잘 먹었습니다 토핑 좋아요
답변   : 안녕하세요 프 라텔 로 입니다 저희 프 라텔 로 를 이용 해주셔서 정말 감사 드립니다 앞 으로도 맛있는 한 끼 책임지겠습니다 자주 이용 부탁드려요
예측답변: 맛있게 드셔 주시 고 시간 내서 소중한 리뷰 달아주셔서 너무 너무 감사 드립니다 고